In [5]:
from dataclasses import dataclass
from typing import Dict, List, Optional
import json
from datetime import datetime

@dataclass
class Gap:
    skill: str
    current: int
    required: int
    gap_size: int
    priority: str
    additional_info: Optional[Dict] = None

class SkillAnalyzer:
    def __init__(self, data_path: str):
        with open(data_path, 'r') as f:
            self.data = json.load(f)
        self.physician = self.data["physician"]
        self.requirements = self.data["role_requirements"]["senior_internist"]
    
    def analyze_clinical_procedures(self) -> List[Gap]:
        gaps = []
        current = self.physician["current_skills"]["clinical_procedures"]
        required = self.requirements["clinical_procedures"]
        
        for proc, req in required.items():
            curr = current.get(proc, {})
            if curr.get("proficiency", 0) < req["min_proficiency"]:
                gaps.append(Gap(
                    skill=proc,
                    current=curr.get("proficiency", 0),
                    required=req["min_proficiency"],
                    gap_size=req["min_proficiency"] - curr.get("proficiency", 0),
                    priority="high" if req["min_proficiency"] - curr.get("proficiency", 0) > 1 else "medium",
                    additional_info={
                        "procedures_gap": req["min_procedures"] - curr.get("procedures_performed", 0),
                        "missing_certs": list(set(req["required_certs"]) - set(curr.get("certifications", [])))
                    }
                ))
        return gaps

    def analyze_patient_care(self) -> List[Gap]:
        gaps = []
        current = self.physician["current_skills"]["patient_care"]
        required = self.requirements["patient_care"]
        
        for skill, req_level in required.items():
            curr_level = current.get(skill, 0)
            if curr_level < req_level:
                gaps.append(Gap(
                    skill=skill,
                    current=curr_level,
                    required=req_level,
                    gap_size=req_level - curr_level,
                    priority="high" if req_level - curr_level > 1 else "medium"
                ))
        return gaps

    def generate_report(self) -> Dict:
        clinical_gaps = self.analyze_clinical_procedures()
        care_gaps = self.analyze_patient_care()
        
        report = {
            "physician_id": self.physician["id"],
            "name": self.physician["name"],
            "analysis_date": datetime.now().strftime("%Y-%m-%d"),
            "gaps": {
                "clinical_procedures": [vars(gap) for gap in clinical_gaps],
                "patient_care": [vars(gap) for gap in care_gaps]
            },
            "recommendations": []
        }
        
        # Add recommendations
        for gap in clinical_gaps + care_gaps:
            if gap.priority == "high":
                report["recommendations"].append(
                    f"URGENT: Improve {gap.skill} from level {gap.current} to {gap.required}"
                )
                if hasattr(gap, 'additional_info') and gap.additional_info:
                    if gap.additional_info["missing_certs"]:
                        report["recommendations"].extend(
                            [f"Obtain certification: {cert}" for cert in gap.additional_info["missing_certs"]]
                        )
        
        return report

    def save_report(self, output_path: str):
        report = self.generate_report()
        with open(output_path, 'w') as f:
            json.dump(report, f, indent=2)

if __name__ == "__main__":
    analyzer = SkillAnalyzer("physician_requirements.json")
    analyzer.save_report("skill_gaps.json")

TypeError: Object of type SkillGap is not JSON serializable